In [4]:
!/usr/local/Cellar/jupyterlab/4.1.6_1/libexec/bin/python -m pip install pymilvus

  Using cached pymilvus-2.4.0-1-py3-none-any.whl.metadata (4.5 kB)
  Using cached setuptools-69.5.1-py3-none-any.whl.metadata (6.2 kB)
  Using cached grpcio-1.60.0-cp312-cp312-macosx_10_10_universal2.whl.metadata (4.0 kB)
  Using cached protobuf-5.26.1-cp37-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached environs-9.5.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached ujson-5.9.0-cp312-cp312-macosx_10_9_x86_64.whl.metadata (8.7 kB)
  Using cached pandas-2.2.2-cp312-cp312-macosx_10_9_x86_64.whl.metadata (19 kB)
  Using cached minio-7.2.5-py3-none-any.whl.metadata (6.4 kB)
  Using cached pyarrow-15.0.2-cp312-cp312-macosx_10_15_x86_64.whl.metadata (3.0 kB)
  Using cached azure_storage_blob-12.19.1-py3-none-any.whl.metadata (26 kB)
  Using cached scipy-1.13.0-cp312-cp312-macosx_10_9_x86_64.whl.metadata (60 kB)
  Using cached marshmallow-3.21.1-py3-none-any.whl.metadata (7.2 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached pycryp

In [7]:
!/usr/local/Cellar/jupyterlab/4.1.6_1/libexec/bin/python -m pip install -q towhee towhee.models gradio

In [4]:
! curl -L https://github.com/towhee-io/examples/releases/download/data/question_answer.csv -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  595k  100  595k    0     0   328k      0  0:00:01  0:00:01 --:--:--  443k


In [5]:
!/usr/local/Cellar/jupyterlab/4.1.6_1/libexec/bin/python -m pip install pandas

In [6]:
import pandas as pd

df = pd.read_csv('question_answer.csv')
df.head()

,id,question,answer
0,0,Is Disability Insurance Required By Law?,Not generally. There are five states that requ...
1,1,Can Creditors Take Life Insurance After ...,If the person who passed away was the one with...
2,2,Does Travelers Insurance Have Renters Ins...,One of the insurance carriers I represent is T...
3,3,Can I Drive A New Car Home Without Ins...,Most auto dealers will not let you drive the c...
4,4,Is The Cash Surrender Value Of Life Ins...,Cash surrender value comes only with Whole Lif...


In [8]:
id_answer = df.set_index('id')['answer'].to_dict()


In [9]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

connections.connect(host='127.0.0.1', port='19530')

def create_milvus_collection(collection_name, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
    
    fields = [
    FieldSchema(name='id', dtype=DataType.VARCHAR, descrition='ids', max_length=500, is_primary=True, auto_id=False),
    FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, descrition='embedding vectors', dim=dim)
    ]
    schema = CollectionSchema(fields=fields, description='reverse image search')
    collection = Collection(name=collection_name, schema=schema)

    # create IVF_FLAT index for collection.
    index_params = {
        'metric_type':'L2',
        'index_type':"IVF_FLAT",
        'params':{"nlist":2048}
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    return collection

collection = create_milvus_collection('question_answer', 768)

In [12]:
%%time
from towhee import pipe, ops
import numpy as np
from towhee.datacollection import DataCollection

insert_pipe = (
    pipe.input('id', 'question', 'answer')
        .map('question', 'vec', ops.text_embedding.dpr(model_name='facebook/dpr-ctx_encoder-single-nq-base'))
        .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
        .map(('id', 'vec'), 'insert_status', ops.ann_insert.milvus_client(host='127.0.0.1', port='19530', collection_name='question_answer'))
        .output()
)

import csv
with open('question_answer.csv', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        insert_pipe(*row)
print(i)

CPU times: user 1min 45s, sys: 51 s, total: 2min 36s
Wall time: 55.9 s


In [13]:
print('Total number of inserted data is {}.'.format(collection.num_entities))


Total number of inserted data is 0.


In [19]:
%%time
collection.load()
ans_pipe = (
    pipe.input('question')
        .map('question', 'vec', ops.text_embedding.dpr(model_name="facebook/dpr-ctx_encoder-single-nq-base"))
        .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
        .map('vec', 'res', ops.ann_search.milvus_client(host='127.0.0.1', port='19530', collection_name='question_answer', limit=1))
        .map('res', 'answer', lambda x: [id_answer[int(i[0])] for i in x])
        .output('question', 'answer')
)


ans = ans_pipe('Law')
ans = DataCollection(ans)
ans.show()

question,answer
Law,Renters insurance does not cover damage to vehicles. The vehicles insurance would cover what ever damage occurred. There is a sl...


CPU times: user 818 ms, sys: 1.18 s, total: 2 s
Wall time: 2.18 s
